# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor
from PositionalModel import PositionalModel

from src.match_names import *
from config import *

In [3]:
info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "Price"]
to_predict = ["FPL_points"]

In [4]:
len(FEATURES_OUTFIELD)

42

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")
CURRENT_SEASON = "2023-24"

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (2,3,6,8,36,43,44,45,59,84,86,147,208) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df.shape

(33970, 274)

In [7]:
df.columns

Index(['Unnamed: 0', 'Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name',
       'Start', 'Pos', 'Min',
       ...
       'FPL_xGC_30', 'Opp_rating_30', 'Team_rating_30', 'Min_points_30',
       'Team_result_30', 'DEF', 'FWD', 'GK', 'MID', 'Finished'],
      dtype='object', length=274)

In [8]:
# df = df[df["GW"] == 2]

In [9]:
# X_test = df[df["Season"] == CURRENT_SEASON][FEATURES_OUTFIELD]
df = df[df["Season"] == CURRENT_SEASON]
# .reset_index(drop=True)

In [10]:
df.isna().sum().sort_values(ascending=False).head(20)

FPL_GW           16200
Min_points       16200
Clean_name       16200
Min_points_2     16200
Min_points_4     16200
Min_points_30    16200
Cmp%             15695
SCA              15691
Blocks           15691
T_succ           15691
T_att            15691
PrgC             15691
Carries          15691
PrgP             15691
Att              15691
Cmp              15691
GCA              15691
xA               15691
npxG             15691
xG               15691
dtype: int64

In [11]:
# df[FEATURES_OUTFIELD].iloc[0]

In [12]:
df = df[~df[FEATURES_OUTFIELD].isnull().any(axis=1)] # for some reason 45 rows are missing some feature values

In [13]:
df.shape

(16200, 274)

In [14]:
X_test = df.copy()

In [15]:
df.columns

Index(['Unnamed: 0', 'Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name',
       'Start', 'Pos', 'Min',
       ...
       'FPL_xGC_30', 'Opp_rating_30', 'Team_rating_30', 'Min_points_30',
       'Team_result_30', 'DEF', 'FWD', 'GK', 'MID', 'Finished'],
      dtype='object', length=274)

In [16]:
file = open("models/GBR_positional.pkl",'rb')
model = pickle.load(file)

In [17]:
model

In [18]:
model.predict( X_test.head(5) )

[2.675017836974815,
 2.727293756998658,
 2.0966502567059826,
 3.0970239373076933,
 4.251298349293246]

In [19]:
model.predict( X_test.head(3) )

[2.675017836974815, 2.727293756998658, 2.0966502567059826]

In [20]:
model.predict( X_test.head(4) )

[2.675017836974815, 2.727293756998658, 2.0966502567059826, 3.0970239373076933]

In [21]:
X_test.head(5)

,Unnamed: 0,Date,Day,Venue,Team,Opponent,Name,Start,Pos,Min,...,FPL_xGC_30,Opp_rating_30,Team_rating_30,Min_points_30,Team_result_30,DEF,FWD,GK,MID,Finished
76,76,2023-08-20,Sun,Home,West Ham,Chelsea,Aaron-Cresswell,0.0,0,0.0,...,1.02,1777.369116,1761.774137,NaN,1.166667,1.0,0.0,0.0,0.0,True
105,105,2023-08-13,Sun,Home,Brentford,Tottenham,Aaron-Hickey,1.0,RB,71.0,...,1.02,1773.661752,1765.611925,NaN,1.428571,1.0,0.0,0.0,0.0,True
106,106,2023-08-19,Sat,Away,Brentford,Fulham,Aaron-Hickey,1.0,RB,77.0,...,1.02,1775.117326,1767.636702,NaN,1.413793,1.0,0.0,0.0,0.0,True
181,181,2023-08-12,Sat,Home,Arsenal,Nott'ham Forest,Aaron-Ramsdale,1.0,GK,90.0,...,1.18,1780.652535,1925.740381,NaN,2.100000,0.0,0.0,1.0,0.0,True
182,182,2023-08-21,Mon,Away,Arsenal,Crystal Palace,Aaron-Ramsdale,1.0,GK,90.0,...,1.18,1770.298551,1926.720374,NaN,2.100000,0.0,0.0,1.0,0.0,True


In [22]:
X_test.head(3)

,Unnamed: 0,Date,Day,Venue,Team,Opponent,Name,Start,Pos,Min,...,FPL_xGC_30,Opp_rating_30,Team_rating_30,Min_points_30,Team_result_30,DEF,FWD,GK,MID,Finished
76,76,2023-08-20,Sun,Home,West Ham,Chelsea,Aaron-Cresswell,0.0,0,0.0,...,1.02,1777.369116,1761.774137,NaN,1.166667,1.0,0.0,0.0,0.0,True
105,105,2023-08-13,Sun,Home,Brentford,Tottenham,Aaron-Hickey,1.0,RB,71.0,...,1.02,1773.661752,1765.611925,NaN,1.428571,1.0,0.0,0.0,0.0,True
106,106,2023-08-19,Sat,Away,Brentford,Fulham,Aaron-Hickey,1.0,RB,77.0,...,1.02,1775.117326,1767.636702,NaN,1.413793,1.0,0.0,0.0,0.0,True


# Predicting

In [23]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [24]:
def adjust_goalkeepers(df):
    gks = pd.read_csv("data/misc/goalkeepers.csv")
    gk_names = gks["Name"].to_list()
    gk_names = [name_fbref_to_fpl(neutralize_name(n)) for n in gk_names]
    # print(gk_names)
    df.loc[(df["FPL_pos"] == "GK") & ~(df["Name_original"].apply(neutralize_name).isin(gk_names)), "Pred"] = 0
    return df

In [25]:
def get_predictions(model, df, X):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    print(len(predictions), df_predictions.shape)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_goalkeepers(preds)
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [26]:
X_test[FEATURES_OUTFIELD].isna()

,Was_home,Rating_difference,Price,Transfers_balance,Avg_FPL_points,Min_2,Gls_2,Ast_2,xG_2,xA_2,...,Team_CS_30,Team_score_30,Opp_score_30,Team_result_30,Player_GC_30,FPL_points_30,DEF,FWD,GK,MID
76,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
105,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
106,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
181,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
182,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33965,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
33966,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
33967,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
33968,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [27]:
# X_test.shape,  model.predict(X_test).shape

In [28]:
%%time
preds = get_predictions(model, df, X_test)

16200 (16200, 274)
CPU times: total: 2.19 s
Wall time: 727 ms


C:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [29]:
# next gameweek
preds[preds["GW"] == NEXT_GAMEWEEK].head(50)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
5179,Erling Haaland,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,FWD,14.0,7.812250
6680,Jack Grealish,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,MID,7.4,6.238789
11631,Mohamed Salah,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,MID,12.5,6.185346
2803,Callum Wilson,4.0,2023-24,Newcastle Utd,Brighton,0.0,1884.512817,1843.758179,FWD,7.9,6.130820
5981,Guglielmo Vicario,4.0,2023-24,Tottenham,Burnley,0.0,1829.239014,1722.952881,GK,5.0,5.933283
4690,Eddie Nketiah,4.0,2023-24,Arsenal,Manchester Utd,1.0,1927.032104,1865.755249,FWD,5.6,5.618999
13759,Rúben Gato Alves Dias,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,DEF,5.5,5.413179
1127,Alexis Mac Allister,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,MID,6.0,5.368156
13550,Rodrigo Hernandez,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,MID,5.6,5.258674
12854,Phil Foden,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,MID,7.6,5.252377


In [30]:
preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == NEXT_GAMEWEEK)].head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
5981,Guglielmo Vicario,4.0,2023-24,Tottenham,Burnley,0.0,1829.239014,1722.952881,GK,5.0,5.933283
12154,Nick Pope,4.0,2023-24,Newcastle Utd,Brighton,0.0,1884.512817,1843.758179,GK,5.5,5.071843
13793,Sam Johnstone,4.0,2023-24,Crystal Palace,Wolves,1.0,1758.725586,1704.537109,GK,4.5,4.979079
4725,Ederson Santana de Moraes,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,GK,5.5,4.354725
8323,Jordan Pickford,4.0,2023-24,Everton,Sheffield United,0.0,1691.463989,1631.764282,GK,4.5,3.939942
1197,Alisson Ramses Becker,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,GK,5.5,3.837491
707,Aaron Ramsdale,4.0,2023-24,Arsenal,Manchester Utd,1.0,1927.032104,1865.755249,GK,5.0,3.702781
7414,Jason Steele,4.0,2023-24,Brighton,Newcastle Utd,1.0,1843.758179,1884.512817,GK,4.5,3.678688
13445,Robert Sánchez,4.0,2023-24,Chelsea,Nott'ham Forest,1.0,1781.043579,1677.877930,GK,4.5,3.385826
12259,Norberto Murara Neto,4.0,2023-24,Bournemouth,Brentford,0.0,1660.290894,1835.573242,GK,4.5,3.095135


In [31]:
# all preds
preds.head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
5184,Erling Haaland,9.0,2023-24,Manchester City,Brighton,1.0,2083.702148,1843.758179,FWD,14.0,7.812250
5198,Erling Haaland,24.0,2023-24,Manchester City,Everton,1.0,2083.702148,1691.463989,FWD,14.0,7.812250
5212,Erling Haaland,38.0,2023-24,Manchester City,West Ham,1.0,2083.702148,1787.825562,FWD,14.0,7.812250
5192,Erling Haaland,17.0,2023-24,Manchester City,Crystal Palace,1.0,2083.702148,1758.725586,FWD,14.0,7.812250
5196,Erling Haaland,22.0,2023-24,Manchester City,Burnley,1.0,2083.702148,1722.952881,FWD,14.0,7.812250
5189,Erling Haaland,14.0,2023-24,Manchester City,Tottenham,1.0,2083.702148,1829.239014,FWD,14.0,7.812250
5210,Erling Haaland,36.0,2023-24,Manchester City,Wolves,1.0,2083.702148,1704.537109,FWD,14.0,7.812250
5188,Erling Haaland,13.0,2023-24,Manchester City,Liverpool,1.0,2083.702148,1946.521606,FWD,14.0,7.812250
5186,Erling Haaland,11.0,2023-24,Manchester City,Bournemouth,1.0,2083.702148,1660.290894,FWD,14.0,7.812250
5207,Erling Haaland,33.0,2023-24,Manchester City,Luton,1.0,2083.702148,1602.477417,FWD,14.0,7.812250


In [32]:
preds[preds["FPL_pos"] == "GK"].head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
5992,Guglielmo Vicario,15.0,2023-24,Tottenham,West Ham,1.0,1829.239014,1787.825562,GK,5.0,6.163241
5986,Guglielmo Vicario,9.0,2023-24,Tottenham,Fulham,1.0,1829.239014,1732.639404,GK,5.0,6.163241
5990,Guglielmo Vicario,13.0,2023-24,Tottenham,Aston Villa,1.0,1829.239014,1825.741455,GK,5.0,6.163241
6014,Guglielmo Vicario,37.0,2023-24,Tottenham,Burnley,1.0,1829.239014,1722.952881,GK,5.0,6.163241
5988,Guglielmo Vicario,11.0,2023-24,Tottenham,Chelsea,1.0,1829.239014,1781.043579,GK,5.0,6.163241
6009,Guglielmo Vicario,32.0,2023-24,Tottenham,Nott'ham Forest,1.0,1829.239014,1677.877930,GK,5.0,6.163241
5995,Guglielmo Vicario,18.0,2023-24,Tottenham,Everton,1.0,1829.239014,1691.463989,GK,5.0,6.163241
6004,Guglielmo Vicario,27.0,2023-24,Tottenham,Crystal Palace,1.0,1829.239014,1758.725586,GK,5.0,6.163241
6002,Guglielmo Vicario,25.0,2023-24,Tottenham,Wolves,1.0,1829.239014,1704.537109,GK,5.0,6.163241
5997,Guglielmo Vicario,20.0,2023-24,Tottenham,Bournemouth,1.0,1829.239014,1660.290894,GK,5.0,6.163241


In [33]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
Erling Haaland,263.255733
Callum Wilson,221.561804
Guglielmo Vicario,218.223096
Jack Grealish,213.021143
Mohamed Salah,212.457691
Solly March,208.102221
Bruno Borges Fernandes,197.381246
Miguel Almirón Rejala,192.304859
Eddie Nketiah,188.374220


# Saving to file

In [34]:
preds.to_csv("predictions/preds_next_season.csv", index=False)